# Introduction to atomman: Free Surface Rotation Basis

__Lucas M. Hale__, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), _Materials Science and Engineering Division, NIST_.
    
[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 

## 1. Introduction

This Notebook outlines the free_surface_basis function which determines the appropriate rotational \[uvw\] Miller vectors for generating a free surface configuration.  

*Added version 1.2.7*

**Library imports**

In [1]:
# Standard Python libraries
import datetime

# http://www.numpy.org/
import numpy as np

# https://github.com/usnistgov/atomman
import atomman as am
import atomman.unitconvert as uc

# Show atomman version
print('atomman version =', am.__version__)

# Show date of Notebook execution
print('Notebook executed on', datetime.date.today())

atomman version = 1.3.0
Notebook executed on 2019-11-05


## 2. Theory

Atomic simulations of free surfaces are typically evaluated in the following way:

1. A bulk atomic system is constructed such that the crystal plane of interest coincides with one of the system's boundaries.

2. The cohesive energy of the bulk system is evaluated with all boundary conditions periodic.

3. The free surface is inserted by either making one of the boundaries non-periodic, or greatly expanding said boundary dimension. 

Coordinate system vectors:

- $a_{unit}, b_{unit}, c_{unit}$ refer to the lattice vectors of a standard crystal unit cell.

- $a_{box}, b_{box}, c_{box}$ refer to the box vectors of the full atomic system.

- $[u_a, v_a, w_a], [u_b, v_b, w_b], [u_c, v_c, w_c]$ are the crystallographic Miller \[uvw\] indices coinciding with the three system box vectors. 

The unit and box vectors can be related using the \[uvw\] indices as:
        
$$ a_{box} = u_a a_{unit} + v_a b_{unit} + w_a c_{unit}$$
$$ b_{box} = u_b a_{unit} + v_b b_{unit} + w_b c_{unit}$$
$$ c_{box} = u_c a_{unit} + v_c b_{unit} + w_c c_{unit}$$

To ensure the generated system is periodic for calculation step \#2, the box vectors must be full periodic lattice steps. In other words, all the hkl's must be integers. 

Generating the atomic system (step \#1) requires that two of the system's box vectors be contained within the slip plane of interest. Note that this is *not* the same as taking one of the box vectors to be perpendicular to the slip plane. [Sun and Cedar](https://doi.org/10.1016/j.susc.2013.05.016) showed that this could be done by identifying three points within the same slip plane, and using the distance vectors between pairs of those points to obtain the two box vectors. The Miller (hkl) plane indices indicate the intercepts of the crystallographic plane relative to the unit cell, which can be used to obtain the three points:

- An (hkl) plane has intercepts at $\frac{a_{unit}}{h}$, $\frac{b_{unit}}{k}$ and $\frac{c_{unit}}{l}$. 

- An (hk0) plane has the intercepts $\frac{a_{unit}}{h}$ and $\frac{b_{unit}}{k}$, and a third point can be obtained by adding $c_{unit}$ to either intercept.  The (h0l) and (0kl) planes can be similarly obtained.

- An (h00) plane has an intercept $\frac{a_{unit}}{h}$, and two other points can be obtained by adding either $b_{unit}$ or $c_{unit}$ to the intercept.  The (0k0) and (00l) planes can be similarly obtained.

As the two resulting in-plane vectors are in terms of the unit cell vectors, they can be written solely in terms of \[uvw\] indices (and therefore system-independent).  For simplicity, we'll take $a_{box}$ and $b_{box}$ to be the in-plane vectors.

- (hkl):
    
    - $[u_a, v_a, w_a] = [\frac{-M}{h}, \frac{M}{k}, 0]$
    - $[u_b, v_b, w_b] = [\frac{-M}{h}, 0, \frac{M}{l}]$

- (hk0):

    - $[u_a, v_a, w_a] = [\frac{-M}{h}, \frac{M}{k}, 0]$
    - $[u_b, v_b, w_b] = [0, 0, 1]$
    
- (h00):

    - $[u_a, v_a, w_a] = [0, 1, 0]$
    - $[u_b, v_b, w_b] = [0, 0, 1]$
    
where $M$ is a multiplier to ensure the \[uvw\]'s are integers.

The $c_{box}$ can be any vector not in the slip plane (which it can't be for the system to have volume!). Practically, it should be defined such that $[u_c, v_c, w_c]$ are small integers and that $c_{box}$ is close to the slip plane normal.

**Note**: In LAMMPS, the resulting system is transformed to adhere to the box vectors limitations:

- $a'_{box} = [lx, 0, 0]$

- $b'_{box} = [xy, ly, 0]$

- $c'_{box} = [xz, yz, lz]$

This transformation results in the free surface plane being normal to the Cartesian z-axis as only the $c'_{box}$ vector has a z-component.

## 3. Function Documentation

Generates the uvw box vector orientations for a free surface atomic system.  In determining the uvw sets, two sets will be in the specified hkl plane and one will be out of the plane.  Uses free surface in-plane vector determination algorithm by [Sun and Cedar](https://doi.org/10.1016/j.susc.2013.05.016) to identify two in-plane vectors and the plane normal. The shortest in-plane vector is identified, as well as an out-of-plane vector close to the plane normal.  The second in-plane vector is then selected to be a shortest in-plane vector that is not parallel to the first.
    
Parameters

- __hkl__ (*array-like object*) The free surface plane to generate expressed in either 3 indices Miller (hkl) format or 4 indices Miller-Bravais (hkil) format.

- __box__ (*atomman.Box, optional*)  The box object associated with the unit cell. Used to identify the best uvw set for the out-of-plane box vector.  Default value uses a cubic box.

- __cutboxvector__ (*str, optional*) Specifies which of the three box vectors corresponds to the out-of-plane vector.  Default value is c.

- __n__ (*int, optional*) Max uvw index value to use in identifying the best uvw set for the out-of-plane vector.  If not given, will use the largest absolute index between the given hkl and the initial in-plane vector guesses.

- __return_hexagonal__ (*bool, optional*) Flag for indicating if the returned vectors are expressed in Miller [uvw] format (False) or Miller-Bravais [uvtw] format (True).  The Miller-Bravais format is only allowed if box is in the standard hexagonal setting: a=b!=c, alpha=beta=90, gamma=120.  Default value is False if hkl is given in the 3 indices Miller (hkl) format and True if it is given in the 4 indices Miller-Bravais (hkil) format.
    
- __return_planenormal__ (*bool, optional*) If True, the computed Cartesian plane normal will also be returned.  Default value is False.

- __conventional_setting__ (*str, optional*) Allows for rotations of a primitive unit cell box to be determined from (hkl) indices specified relative to a conventional unit cell.  Allowed settings: 'p' for primitive (no conversion), 'f' for face-centered, 'i' for body-centered, and 'a', 'b', or 'c' for side-centered.  Default behavior is to perform no conversion, i.e. take (hkl) relative to the given box.
    
Returns

- __uvws__ (*numpy.ndarray*) 3x3 array of Miller \[uvw\] vectors or 3x4 array of Miller-Bravais \[uvtw\] vectors to rotate the unit cell for a free surface configuration.

- __planenormal__ (*numpy.ndarray*) The Cartesian plane normal vector.  Only returned if return_planenormal is True.
        
Raises

- (*ValueError*) If invalid hkl indices values are given.
- (*AssertionError*) If the search fails to find any of the three \[uvw\] rotation vectors.

## 4. Examples

### 4.1. Cubic

In [2]:
a = uc.set_in_units(2.8665, 'angstrom')
box = am.Box.cubic(a)

In [3]:
plane = [0, 0, 1]

uvws = am.defect.free_surface_basis(plane, box=box)
print('a_uvw =', uvws[0])
print('b_uvw =', uvws[1])
print('c_uvw =', uvws[2])

a_uvw = [1. 0. 0.]
b_uvw = [0. 1. 0.]
c_uvw = [0. 0. 1.]


In [4]:
plane = [1, 1, 0]

uvws = am.defect.free_surface_basis(plane, box=box)
print('a_uvw =', uvws[0])
print('b_uvw =', uvws[1])
print('c_uvw =', uvws[2])

a_uvw = [0. 0. 1.]
b_uvw = [ 1. -1.  0.]
c_uvw = [1. 1. 0.]


In [5]:
plane = [-1, -1, -2]

uvws = am.defect.free_surface_basis(plane, box=box)
print('a_uvw =', uvws[0])
print('b_uvw =', uvws[1])
print('c_uvw =', uvws[2])

a_uvw = [-1.  1.  0.]
b_uvw = [ 1.  1. -1.]
c_uvw = [-1. -1. -2.]


In [6]:
plane = [1, -2, 3]

uvws = am.defect.free_surface_basis(plane, box=box)
print('a_uvw =', uvws[0])
print('b_uvw =', uvws[1])
print('c_uvw =', uvws[2])

a_uvw = [-1.  1.  1.]
b_uvw = [-2. -1.  0.]
c_uvw = [ 1. -2.  3.]


### 4.2 Hexagonal

In [7]:
a = uc.set_in_units(3.254, 'angstrom')
c = uc.set_in_units(5.63, 'angstrom')
box = am.Box.hexagonal(a, c)

In [8]:
plane = [0, 0, 0, 1]

uvtws = am.defect.free_surface_basis(plane, box=box)
uvws = am.defect.free_surface_basis(plane, box=box, return_hexagonal=False)

print('a_uvtw = 1/3', uvtws[0] * 3)
print('b_uvtw = 1/3', uvtws[1] * 3)
print('c_uvtw = 1/3', uvtws[2] * 3)
print()
print('a_uvw =', uvws[0])
print('b_uvw =', uvws[1])
print('c_uvw =', uvws[2])

a_uvtw = 1/3 [-1.  2. -1.  0.]
b_uvtw = 1/3 [-2.  1.  1.  0.]
c_uvtw = 1/3 [ 0.  0. -0.  3.]

a_uvw = [0. 1. 0.]
b_uvw = [-1.  0.  0.]
c_uvw = [0. 0. 1.]


In [9]:
plane = [1, 0, -1, 0]

uvtws = am.defect.free_surface_basis(plane, box=box)
uvws = am.defect.free_surface_basis(plane, box=box, return_hexagonal=False)

print('a_uvtw = 1/3', uvtws[0] * 3)
print('b_uvtw = 1/3', uvtws[1] * 3)
print('c_uvtw = 1/3', uvtws[2] * 3)
print()
print('a_uvw =', uvws[0])
print('b_uvw =', uvws[1])
print('c_uvw =', uvws[2])

a_uvtw = 1/3 [-1.  2. -1.  0.]
b_uvtw = 1/3 [ 0.  0. -0.  3.]
c_uvtw = 1/3 [ 2. -1. -1.  0.]

a_uvw = [0. 1. 0.]
b_uvw = [0. 0. 1.]
c_uvw = [1. 0. 0.]


In [10]:
plane = [1, 0, -1, 1]

uvtws = am.defect.free_surface_basis(plane, box=box)
uvws = am.defect.free_surface_basis(plane, box=box, return_hexagonal=False)

print('a_uvtw = 1/3', uvtws[0] * 3)
print('b_uvtw = 1/3', uvtws[1] * 3)
print('c_uvtw = 1/3', uvtws[2] * 3)
print()
print('a_uvw =', uvws[0])
print('b_uvw =', uvws[1])
print('c_uvw =', uvws[2])

a_uvtw = 1/3 [-1.  2. -1.  0.]
b_uvtw = 1/3 [-2.  1.  1.  3.]
c_uvtw = 1/3 [ 2. -1. -1.  3.]

a_uvw = [0. 1. 0.]
b_uvw = [-1.  0.  1.]
c_uvw = [1. 0. 1.]


### 4.3 Using primitive cells

For DFT calculations, it is useful to use primitive unit cells rather than conventional ones to minimize the resulting system size.  However, this introduces a complication that the Miller (hkl) plane and \[uvw\] vectors specified relative to a conventional unit cell will not work on the corresponding primitive without conversions.  The conventional_setting parameter allows for the (hkl) to be given relative to a conventional cell and for the rotations to be computed for the corresponding primitive cell.

In [11]:
# Construct a box for a primitive bcc unit cell
a = uc.set_in_units(2.866, 'angstrom')
box = am.Box.trigonal(a = a * 3**0.5 / 2, alpha = 109.47122063449069)

In [12]:
# Plane defined relative to conventional bcc cell
plane = [1,1,0]
setting = 'i'

uvws = am.defect.free_surface_basis(plane, box=box, conventional_setting=setting)
print('primitive uvws:')
print('a_uvw =', uvws[0])
print('b_uvw =', uvws[1])
print('c_uvw =', uvws[2])

# Check that conventional uvws are as expected
conventional_uvws = am.tools.miller.vector_primitive_to_conventional(uvws, setting=setting)
print('conventional uvws:')
print('a_uvw =', conventional_uvws[0])
print('b_uvw =', conventional_uvws[1])
print('c_uvw =', conventional_uvws[2])

primitive uvws:
a_uvw = [0. 1. 0.]
b_uvw = [1. 1. 1.]
c_uvw = [ 1.  0. -1.]
conventional uvws:
a_uvw = [-0.5  0.5 -0.5]
b_uvw = [-0.5  0.5  0.5]
c_uvw = [1. 1. 0.]
